Generating plots that explore the disordered LOPC model. This notebook is for testing the disorder method and looking at the optical properties of individual examples of disordered LOPCs without aggregation.

In [ ]:
# computation
import lumapi
import numpy as np
import warnings
import xarray as xr
import xyzpy as xyz
from xyzpy.gen.combo_runner import multi_concat
from multilayer_simulator.lumerical_classes import LumericalOscillator, format_stackrt, format_stackfield
from multilayer_simulator.helpers.mixins import convert_wavelength_and_frequency
import dask
from functools import partial
# plotting
import hvplot.xarray
import holoviews as hv
from holoviews import dim, opts
import panel as pn
import panel.widgets as pnw

In [ ]:
hv.extension("bokeh", "matplotlib", case_sensitive_completion=True)
pn.config.throttled = True

xarray_engine='h5netcdf'

In [ ]:
# A hacky way to import a script

import os
import sys

dir2 = os.path.abspath("")
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path:
    sys.path.append(dir1)
from LOPC import LOPC
from LOPC.helpers import (
    assign_derived_attrs,
    restack,
    enhancement_factor,
    combo_length,
    estimate_combo_run_time,
    linewidth_calculator,
    lopc_data,
    spectrum,
    normalise_over_dim,
    integrate_da,
    sel_or_integrate,
    find_optimum_coords,
    plot_secondary,
    pre_process_for_plots,
    vlines,
    coordinate_string,
    plot_da,
    plot_var,
    plot_optimum_over_dim,
    plot_field,
    complex_elements,
    fix_bin_labels,
    mean_and_std
)

Turn on auto-archiving of cells and Holoviews outputs. See the user guide [here](https://holoviews.org/user_guide/Exporting_and_Archiving.html).

Might need to install `ipympl`.

In [ ]:
# hv.archive.auto() # This breaks the DynamicMap call below due to something to do with pickling

undisordered_run_number = 2
run_number = 211

Useful variables for plotting later:

In [ ]:
blue = hv.Cycle.default_cycles['default_colors'][0]
red = hv.Cycle.default_cycles['default_colors'][1]
yellow = hv.Cycle.default_cycles['default_colors'][2]
green = hv.Cycle.default_cycles['default_colors'][3]

In [ ]:
wavelengths_in_nanometres = np.linspace(480, 880, 256)
wavelengths = wavelengths_in_nanometres * 1e-9
frequencies = convert_wavelength_and_frequency(wavelengths)
angles = np.linspace(0, 86, 64)

In [ ]:
default_oscillator_params = {
    "N": 1e26,
    "permittivity": 2.2,
    "lorentz_resonance_wavelength": 680,
    "lorentz_linewidth": 7.5e13,
}

# Setup

In [ ]:
# fdtd = lumapi.FDTD()

In [ ]:
# oscillator = LumericalOscillator(session=fdtd)

In [ ]:
# rng = np.random.default_rng(seed=20182023)

In [ ]:
# formatter = format_stackrt(
#     output_format="xarray_dataset",
#     **{
#         "variables": ["Rs", "Rp", "Ts", "Tp"],
#         "add_absorption": False,
#     }
# )

In [ ]:
# resources = {
#     "lumerical_session": fdtd,
#     "oscillator": oscillator,
#     "rng": rng,  # assign seeded rng for reproducibility
#     "simulation_mode": "stackrt",
#     "formatter": formatter,
#     "frequencies": frequencies,
#     "angles": angles,
# }

In [ ]:
# constants = {
#     "apply_disorder": 10,  # True-> only one run; int-> int runs
#     "delta_mode": "pplt",  # abs-> interpret delta as distance in nm; pplt-> interpret delta as proportion of passive layer thickness
#     "disorder_type": "uniform",  # not considering gaussian or other distributions
#     "correlated": True,  # False-> uncorrelated disorder
#     "retain_runs": False,  # True-> return concat'd array; False-> return mean array
#     "add_first_layer": True,  # add dummy passive layer to allow disorder on first excitonic layer
#     "copy_layers": True,
#     "length_scale": 1e-9,
# }

In [ ]:
# r = xyz.Runner(lopc_data, var_names=None, constants=constants, resources=resources)

## LOPC

In [ ]:
# h_lopc = xyz.Harvester(runner=r, data_name=f"data/run_{run_number}/LOPC.nc")

In [ ]:
# %%time
# # check how long one run takes
# lopc_data(**resources, **constants)
# # 9s

## Reference slabs

The reference slabs are just those from the equivalent undisordered run.

# Testing

## Perturbing

In [ ]:
from LOPC.helpers import visualise_multilayer, thicknesses_to_vspans

In [ ]:
from LOPC.LOPC import adjust_layer_thickness, shift_layer_position

In [ ]:
# expect a warning
lopc = LOPC.LOPC(**resources, **constants, passive_layer_thickness=200, excitonic_layer_thickness=50, remove_last_layer=False)

In [ ]:
lopc.structure.layers

In [ ]:
p_layer = lopc.structure.layers[2]

In [ ]:
adjust_layer_thickness(p_layer, 1e-7)

In [ ]:
p_layer

In [ ]:
e_layer = lopc.structure.layers[1]

In [ ]:
adjust_layer_thickness(e_layer, 5e-8)

In [ ]:
e_layer

In [ ]:
visualise_multilayer(lopc.structure).opts(opts.VSpan(apply_ranges=True, color=blue, width=400))

In [ ]:
shift_layer_position(lopc.structure, layer_index=7, delta=1e-7, compensate=True)

In [ ]:
visualise_multilayer(lopc.structure).opts(opts.VSpan(apply_ranges=True, color=blue, width=400))

In [ ]:
shift_layer_position(lopc.structure, layer_index=11, delta=1e-7, compensate=False)

In [ ]:
visualise_multilayer(lopc.structure).opts(opts.VSpan(apply_ranges=True, color=blue, width=400))

In [ ]:
shift_layer_position(lopc.structure, layer_index=13, delta=-4e-7, compensate=True)

In [ ]:
visualise_multilayer(lopc.structure).opts(opts.VSpan(apply_ranges=True, color=blue, width=400))

In [ ]:
lopc.structure.layers

In [ ]:
excitonic_layers = lopc.structure.layers[1:-1:2]
excitonic_layers

In [ ]:
list(range(1, len(lopc.structure.layers)-1, 2))

In [ ]:
lopc.structure.layers[20]

In [ ]:
a = not 0

In [ ]:
a

## Random

In [ ]:
np.random.seed(123)
np.random.rand()  # deprecated

In [ ]:
rng = np.random.default_rng(123)
rng.random()

In [ ]:
rng = np.random.default_rng(123)
np.random.default_rng(rng).random()  # test: passing a generator to .default_rng should return the same generator

In [ ]:
rng == np.random.default_rng(rng)

In [ ]:
rng == np.random.default_rng(123)

In [ ]:
rng2 = np.random.default_rng(123)

In [ ]:
rng.random(10)[:-1] == rng2.random(10)[1:]  # the next 9 outputs of the first generator are the 2nd-10th output of the fresh one

In [ ]:
%%timeit

[rng.uniform(-1, 1) for _ in range(0, 10)]

In [ ]:
%%timeit

rng.uniform(-1, 1, 10)

In [ ]:
[rng.uniform(-1, 1) for _ in range(0, 10)]

In [ ]:
rng.uniform(-1, 1, 10)

# Data generation

In [ ]:
deltas = [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.11, 0.12, 0.13, 0.14, 0.15]#, 0.2, 0.25, 0.3, 0.4, 0.5]

In [ ]:
combos = {
    "passive_RI": [1.35],
    "incident_medium_RI": [1.35],
    "exit_medium_RI": [1.35],
    "N": [1e26],
    "permittivity": [2.2],
    "lorentz_resonance_wavelength": [680],
    "lorentz_linewidth": [7.5e13],
    "num_periods": np.arange(5, 30, 5),
    "passive_layer_thickness": np.arange(150, 301, 10),
    "excitonic_layer_thickness": np.arange(10, 81, 10),
    "remove_last_layer": [0], # True/False are not compatible with netCDF format
}

In [ ]:
combos_with_delta = {d: combos|{"delta": [d]} for d in deltas}

In [ ]:
# combos_with_delta[0.1]

## Layer thicknesses

### LOPC simulation

In [ ]:
for k, v in combos_with_delta[0.1].items():
    print(f"{k}: length {len(v)}")

In [ ]:
estimate_combo_run_time(9, combos)/3600

In [ ]:
# rngs = [np.random.default_rng(seed=30182023102+i) for i in range(15)]

In [ ]:
# runners = [
#     xyz.Runner(
#         lopc_data,
#         var_names=None,
#         constants=constants,
#         resources=resources | {"rng": rngs[i]},
#     )
#     for i in range(15)
# ]

In [ ]:
# harvesters = [xyz.Harvester(runner=runners[i], data_name=f"data/run_{run_number}/LOPC_00{i+1}.nc") for i in range(15)]

In [ ]:
# combos_with_delta[deltas[1+4]]

In [ ]:
# # only completed the first loop
# with warnings.catch_warnings():
#     warnings.filterwarnings("ignore", "Can't deepcopy")
#     for i in range(5):
#         harvesters[i].harvest_combos(combos_with_delta[deltas[i+4]])

In [ ]:
# with warnings.catch_warnings():
#     warnings.filterwarnings("ignore", "Can't deepcopy")
#     for i in range(1, 15):
#         harvesters[i].harvest_combos(combos_with_delta[deltas[i]])

How embarrasing, I forgot that `np.arange(5, 30, 5)` doesn't include 30 itself, so I need to do those runs separately:

In [ ]:
# rngs = [np.random.default_rng(seed=301820231211+i) for i in range(len(deltas))]

In [ ]:
# runners = [
#     xyz.Runner(
#         lopc_data,
#         var_names=None,
#         constants=constants,
#         resources=resources | {"rng": the_rng},
#     )
#     for the_rng in rngs
# ]

In [ ]:
# with warnings.catch_warnings():
#     warnings.filterwarnings("ignore", "Can't deepcopy")
#     for r, d in zip(runners, deltas):
#         h_lopc = xyz.Harvester(runner=r, data_name=f"data/run_{run_number}/LOPC_N30_{str(d).replace('.', '')}.nc")
#         h_lopc.harvest_combos(combos|{"num_periods": [30], "delta": [d]})

In [ ]:
# ds = h_lopc.full_ds.copy()

### Load LOPC dataset

In [ ]:
# # chunks for per-angle plots
# chunks = {
#     "frequency": 256,
#     "excitonic_layer_thickness": 16,
#     "passive_layer_thickness": 32,
#     "theta": 1,
#     "num_periods": 16,
# }

In [ ]:
# chunks for plotting or integrating over angle
chunks = {
    "frequency": 256,
    "excitonic_layer_thickness": 16,
    "passive_layer_thickness": 32,
    "theta": 16,
    "num_periods": 1,
}

#### Maths to work out good chunk sizes

In [ ]:
def chunk_size(chunks):
    return np.prod(list(chunks.values()))

In [ ]:
# Number of chunks
(combo_length(combos)*256*64) / chunk_size(chunks)

In [ ]:
# Do chunks contain at least a million data points?
chunk_size(chunks)/1e6

####

In [ ]:
ds = xr.open_mfdataset(
    f"data/run_{run_number}/LOPC.nc",
    engine=xarray_engine,
    lock=False,
    # chunks=chunks,
)

# ds.equals(h_lopc.full_ds)

In [ ]:
# add derived attrs
ds = assign_derived_attrs(ds, per_oscillator=["Rs", "Rp", "R", "Ts", "Tp", "T", "As", "Ap", "A"])
# ds

In [ ]:
restack_plt_to_period = partial(
    restack,
    start_idxs=["passive_layer_thickness", "excitonic_layer_thickness"],
    end_idxs=["period", "excitonic_layer_thickness"],
)

In [ ]:
restacked_ds = restack_plt_to_period(ds)

In [ ]:
restacked_ds = pre_process_for_plots(restacked_ds, strict=False)

In [ ]:
ds[["R", "T", "A"]].squeeze().hvplot(x="wavelength")